In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import particle_builder as pb
import sam3d as s3d
import open3d as o3d
from psdframe import Frame
from psdstaticdataset import StaticDataset
from pathlib import Path
from matplotlib import pyplot as plt
from segment_anything import build_sam, SamAutomaticMaskGenerator
from util import Voxelize, num_to_natural
from mesh_to_gaussians import batch_triangles_to_splats, splats_to_oriented_discs
import numpy as np
from initializerdefs import SceneSetup

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
import logging
ch = logging.StreamHandler()
pb.logger.setLevel(logging.DEBUG)
pb.logger.addHandler(ch)

In [43]:
DS_PATH = Path("/home/david/projects/embodied_gaussians/datasets/real/multiple1_aruco/modelling/static")
d = StaticDataset(DS_PATH / "transforms.json")
frames = d.frames
mask_generator = SamAutomaticMaskGenerator(build_sam(checkpoint=pb.sam_checkpoint).to(device="cuda"))
voxelize = Voxelize(voxel_size=pb.VOXEL_SIZE, mode="train", keys=("coord", "color", "group", "normals"))
intermediate_outputs_path = Path("/home/david/projects/SegmentAnything3D/outputs/notebook")
scene = SceneSetup.load(DS_PATH / "scene.pkl")
bbox = pb.get_scene_bounding_box(scene)


Downloading/Locating SAM checkpoint...
Sam checkpoint in located at /home/david/.frame-seg-init/models/sam_vith.pth


In [44]:
pcd_dict = pb.seg_pcd(d, mask_generator, voxelize, bbox=bbox, intermediate_outputs_path=intermediate_outputs_path)


merging 5 point clouds


/home/david/projects/SegmentAnything3D/.venv/lib/python3.10/site-packages/pointops/query.py:19: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:78.)
  idx = torch.cuda.IntTensor(m, nsample).zero_()


merging 3 point clouds
merging 2 point clouds


In [45]:
# create colors from group
group_colors = np.zeros((pcd_dict['group'].shape[0], 3))
unique_groups = np.unique(pcd_dict['group'])
for i, group in enumerate(unique_groups):
    group_colors[pcd_dict['group'] == group] = plt.cm.tab20(i % 20)[:3]

In [46]:

bbox.color = [1.0,0,0]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pcd_dict['coord'])
pcd.colors = o3d.utility.Vector3dVector(group_colors )
o3d.visualization.draw_geometries([pcd,bbox ])

In [43]:
mesh = pb.construct_mesh(pcd_dict)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

[WARNING] /root/Open3D/build/poisson/src/ext_poisson/PoissonRecon/Src/FEMTree.Initialize.inl (Line 193)
          Initialize
          Found bad data: 50


In [48]:
meshes = pb.get_object_meshes(pcd_dict, d, scene)

# iterate over meshes and give each a color
for i, mesh in enumerate(meshes):
    mesh.paint_uniform_color(plt.cm.tab20(i % 20)[:3])

o3d.visualization.draw_geometries(meshes, mesh_show_back_face=True)


Valid groups: [2, 4, 5, 7, 8, 9, 10, 27, 31, 32, 34, 35, 36, 40, 43, 44, 53]
removed : 18 groups
Table group: 7
[WARNING] /root/Open3D/build/poisson/src/ext_poisson/PoissonRecon/Src/FEMTree.Initialize.inl (Line 193)
          Initialize
          Found bad data: 40


In [48]:

clean_meshes = [pb.clean_and_watertight_mesh(mesh) for mesh in meshes]
# iterate over meshes and give each a color
for i, mesh in enumerate(clean_meshes):
    mesh.paint_uniform_color(plt.cm.tab20(i % 20)[:3])
o3d.visualization.draw_geometries(clean_meshes, mesh_show_back_face=True)

In [82]:
workspace_mesh = pb.get_workspace_mesh(scene)
#workspace_mesh.paint_uniform_color([0,0,1])
workspace_mesh.paint_uniform_color([0,1,0])
#workspace_pcd.paint_uniform_color([1,0,0])






o3d.visualization.draw_geometries([workspace_mesh], mesh_show_back_face=True)


Downloading/Locating SAM checkpoint...
Sam checkpoint in located at /home/david/.frame-seg-init/models/sam_vith.pth


In [1]:
workspace_mesh = pb.get_workspace_mesh(scene)
extr_mesh = o3d.t.geometry.TriangleMesh.from_legacy(workspace_mesh).extrude_linear(vector=[0,0,1], scale=0.2, capping=True).to_legacy()
#hull, _ = extr_mesh.compute_convex_hull()
#import pymeshfix
nv, nt = pymeshfix.clean_from_arrays(np.asarray(extr_mesh.vertices), np.asarray(extr_mesh.triangles), joincomp = True, remove_smallest_components=False)
clean_mesh = o3d.geometry.TriangleMesh()
clean_mesh.vertices = o3d.utility.Vector3dVector(nv)
clean_mesh.triangles = o3d.utility.Vector3iVector(nt)

o3d.visualization.draw_geometries([extr_mesh], mesh_show_back_face=True)


NameError: name 'pb' is not defined

In [39]:
table_xyz = scene.ground_gaussians.xyz

table_plane = scene.ground_plane
table_normal = np.array([table_plane[0], table_plane[1], table_plane[2]])
table_pcd_extruded = np.array(table_xyz).copy()

DESIRED_HEIGHT = 1.0
VOXEL_SIZE = 0.05
iters = int(DESIRED_HEIGHT / VOXEL_SIZE)
for i in range(iters):
    new_points = table_xyz + table_normal * VOXEL_SIZE * i
    table_pcd_extruded = np.append(table_pcd_extruded, new_points, axis=0)

table_pcd_extruded = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(table_pcd_extruded))
table_pcd_extruded.paint_uniform_color([0,0,1])

o3d.visualization.draw_geometries([table_pcd_extruded], mesh_show_back_face=True)



In [42]:
table_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(table_xyz))
table_pcd.paint_uniform_color([1,0,0])
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(table_pcd_extruded, VOXEL_SIZE*2)
o3d.visualization.draw_geometries([table_pcd,voxel_grid], mesh_show_back_face=True)



In [59]:
# scene seems busted, ground points are on robot!
# testing background.json

import json

with open(DS_PATH.parent.parent.parent / "empty" / "background.json", "r") as f:
    background = json.load(f)


In [61]:
background_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(background['points']))
background_pcd.paint_uniform_color([1,0,0])

scene_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(scene.ground_gaussians.xyz))
scene_pcd.paint_uniform_color([0,0,1])
o3d.visualization.draw_geometries([background_pcd, scene_pcd], mesh_show_back_face=True)


In [5]:
tblock = meshes[0]
tblock.compute_triangle_normals()
o3d.visualization.draw_geometries([tblock], mesh_show_back_face=True)

In [6]:
triangles = np.asarray(np.asarray(tblock.vertices)[tblock.triangles])
triangle_normals = np.asarray(tblock.triangle_normals)

triangles.shape, triangle_normals.shape

((15680, 3, 3), (15680, 3))

In [7]:
splats = batch_triangles_to_splats(triangles, triangle_normals)

/home/david/projects/SegmentAnything3D/mesh_to_gaussians.py:151: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at ../aten/src/ATen/native/Cross.cpp:62.)
  t = 2.0 * torch.cross(q_xyz, v)


In [8]:
splats['centers'].shape, splats['scales'].shape, splats['rotations'].shape

((15680, 3), (15680, 3), (15680, 4))

In [10]:
discs = splats_to_oriented_discs(splats['centers'], splats['scales'], splats['rotations'])
o3d.visualization.draw_geometries(discs, mesh_show_back_face=True)

In [18]:
box = o3d.geometry.TriangleMesh.create_box(0.5,0.5,0.5)
box.compute_triangle_normals()
triangles = np.asarray(np.asarray(box.vertices)[box.triangles])
triangle_normals = np.asarray(box.triangle_normals)

#mask = triangle_normals[:, 2] == 1  # looks like y axis is borked
#triangle_normals = triangle_normals[mask]
#triangles  = triangles[mask]


splats = batch_triangles_to_splats(triangles, triangle_normals)
discs = splats_to_oriented_discs(splats['centers'], splats['scales'], splats['rotations'])

o3d.visualization.draw_geometries([box, *discs], mesh_show_back_face=True)

KeyboardInterrupt: 

In [55]:
splats_to_oriented_discs(splats['centers'], splats['scales'], splats['rotations'])

[TriangleMesh with 33 points and 32 triangles.,
 TriangleMesh with 33 points and 32 triangles.]

In [53]:
splats['scales']

array([[3.53553391e-01, 1.76776695e-01, 1.00000000e-04],
       [3.53553391e-01, 1.76776695e-01, 1.00000000e-04]])

In [50]:
triangle_normals

array([[0., 1., 0.],
       [0., 1., 0.]])

In [49]:
splats['rotations']

array([[ 0.7071068, -0.7071068,  0.       ,  0.       ],
       [ 0.7071068, -0.7071068,  0.       ,  0.       ]], dtype=float32)

In [39]:
mask = triangle_normals[:, 0] == 0
triangle_normals[mask]

array([[ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0., -1.,  0.],
       [ 0., -1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.]])

In [44]:
from mesh_to_gaussians import create_aligned_ellipsoid
create_aligned_ellipsoid(np.array([[1,0,0]]))


array([[0.7071068, 0.       , 0.7071068, 0.       ]], dtype=float32)

In [45]:
create_aligned_ellipsoid(np.array([[0,1,0]]))

array([[ 0.7071068, -0.7071068,  0.       ,  0.       ]], dtype=float32)

In [46]:
create_aligned_ellipsoid(np.array([[0,0,1]]))

array([[1., 0., 0., 0.]], dtype=float32)

In [47]:
create_aligned_ellipsoid(np.array([[0,-1,0]]))

array([[ 0.7071068,  0.7071068,  0.       , -0.       ]], dtype=float32)

In [51]:

def normal_to_quaternion(normal):
    """
    Convert a normal vector to a rotation quaternion that would rotate [0,0,1] to align with the normal.
    
    Args:
        normal: A numpy array or list containing [x,y,z] coordinates of the normal vector
        
    Returns:
        quaternion: A numpy array [w,x,y,z] representing the rotation quaternion
    """
    # Convert input to numpy array and normalize
    normal = np.array(normal, dtype=float)
    normal = normal / np.linalg.norm(normal)
    
    # Default up vector we're rotating from
    up = np.array([0, 0, 1])
    
    # Get rotation axis and angle
    axis = np.cross(up, normal)
    
    # If normal is parallel to up vector, handle specially
    if np.allclose(axis, 0):
        if np.allclose(normal, up):
            return np.array([1, 0, 0, 0])  # Identity quaternion
        else:
            return np.array([0, 1, 0, 0])  # 180° rotation around X
    
    axis = axis / np.linalg.norm(axis)
    angle = np.arccos(np.dot(up, normal))
    
    # Convert axis-angle to quaternion
    w = np.cos(angle / 2)
    xyz = axis * np.sin(angle / 2)
    
    return np.array([w, xyz[0], xyz[1], xyz[2]])

# Example usage for [0,1,0]
normal = [0, 1, 0]
quaternion = normal_to_quaternion(normal)
print(f"Normal {normal} converted to quaternion [w,x,y,z]: {quaternion}")

Normal [0, 1, 0] converted to quaternion [w,x,y,z]: [ 0.70710678 -0.70710678  0.          0.        ]


In [4]:
DS_PATH = Path("/home/david/projects/embodied_gaussians/datasets/real/multiple1_aruco/modelling/static")
d = StaticDataset(DS_PATH / "transforms.json")
intermediate_outputs_path = Path("/home/david/projects/SegmentAnything3D/outputs/notebook")
scene = SceneSetup.load(DS_PATH / "scene.json")

In [32]:
result = pb.initialize_scene(d, None, intermediate_outputs_path)

merging 5 point clouds
merging 3 point clouds
merging 2 point clouds


Segmented cloud has [ -1   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16
  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70
  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88
  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106
 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124
 125 126 127 128 129 130 131 132 133] unique groups - [ -1   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16
  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70
  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88
  89  90  91  92  93  94  95  96  97  98  99 100 1

Dims: [1 1 1]
Dims: [4 6 2]
Dims: [ 5  7 11]
Dims: [ 7 15  7]
Dims: [1 1 1]
Dims: [1 1 1]
Dims: [1 1 1]
Dims: [1 2 2]
Dims: [1 1 1]
Dims: [1 1 1]
Dims: [1 1 1]
Dims: [1 1 1]
Dims: [2 7 6]
Dims: [5 3 2]
Dims: [7 9 1]
Dims: [1 1 1]
Dims: [6 8 7]
Dims: [4 3 2]
Dims: [1 1 1]
Dims: [1 1 1]
Dims: [2 1 1]
Dims: [5 5 3]
Dims: [4 3 1]
Dims: [1 1 1]
Dims: [5 9 2]
Dims: [5 5 2]
Dims: [1 1 1]
Dims: [6 9 2]
Dims: [1 1 1]
Dims: [109  40   6]
Dims: [2 1 3]
Dims: [1 2 1]
Dims: [1 2 1]
Dims: [1 2 1]
Dims: [2 2 1]
Dims: [ 6  4 10]
Dims: [2 2 4]
Dims: [1 1 1]
Dims: [1 1 2]
Dims: [1 1 1]
Dims: [1 1 1]
Dims: [3 2 1]
Dims: [3 2 1]
Dims: [4 4 1]


In [33]:
len(result.objects)

44

In [43]:
gaussians = result.objects[2].gaussians
particles = result.objects[2].particles

In [41]:
# create spheres from particles, and move to particles position
spheres = [o3d.geometry.TriangleMesh.create_sphere(radius=particle.radius) for particle in particles]
print(len(spheres), len(particles))
for sphere, particle in zip(spheres, particles):
    sphere.translate(particle.xyz)
o3d.visualization.draw_geometries([*spheres], mesh_show_back_face=True)

126 126


In [44]:
# create discs from gaussians, and move to gaussian position
discs = splats_to_oriented_discs(gaussians.xyz, gaussians.scaling, gaussians.rotations, gaussians.colors, resolution=12)
o3d.visualization.draw_geometries(discs, mesh_show_back_face=True)